# Reproduce Inviscid burgers Section

## Data sample figure

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

x = np.load('Data/training data invscid burgers.npy')[:]  # Download data from 
plt.figure(figsize=(10,10));plt.tight_layout();plt.subplot(1,2,1)
plt.imshow(x[1]);plt.xlabel('x');plt.ylabel('t')
plt.subplot(1,2,2);plt.imshow(x[50]);plt.xlabel('x');plt.ylabel('t')
# plt.savefig('burger.svg')

## Error histogram figure

In [ ]:
import pickle
import tensorflow
from utils import *
from tqdm.notebook import tqdm

In [ ]:
#load test data
test_data_finite = np.load('Data/test data invscid burgers finiteNet.npy') 
test_data_weno5 = np.load('Data/test data invscid burgers weno5.npy')
test_data_exact = np.load('Data/test data invscid burgers exact.npy') 
test_data_diffusion = np.load('Data/test data invscid burgers DiffusionNet.npy')

### MSE calculation

In [ ]:
'''
error log => 
    Dictionary [ chunk# ] [ test# ] [ mse_diffuison_exact ]   => MSE between DiffuisonNet prediction and exact solution
    Dictionary [ chunk# ] [ test# ] [ mse_finite_exact ]      => MSE between FiniteNet prediction and exact solution
    Dictionary [ chunk# ] [ test# ] [ mse_weno5_exact ]       => MSE between WENO5 solution and exact solution
    
    chunk := -1 => all chunks error
'''


error_log =  {}
chunks=10
tests=1000

    
for ci in range(chunks):
    indices = np.arange(ci+10,101,10)
    print(f'Chunk#{ci} at {indices}')
    
    error_log_chunk={}
    
    for ti in tqdm(range(tests)):
        
        mse_diffuison_exact=   mse(test_data_diffusion[ti,indices],test_data_exact[ti,indices])
        mse_finite_exact =     mse(test_data_finite[ti,indices],test_data_exact[ti,indices])
        mse_weno5_exact=       mse(test_data_weno5[ti,indices],test_data_exact[ti,indices])
    
    
    
        error_log_chunk[ti]={'mse_diffusion_exact' : mse_diffuison_exact,
                             'mse_finite_exact'    : mse_finite_exact,
                             'mse_weno5_exact'     : mse_weno5_exact}
        
        
    error_log[ci] = error_log_chunk

    
error_log_all={}
print("Combined error of all chunks")
for ti in tqdm(range(1000)):
    error_log_all[ti]=  { 'mse_diffusion_exact' : np.sum([error_log[ci][ti]['mse_diffusion_exact'] for ci in range(10)]) , 
                          'mse_finite_exact'    : np.sum([error_log[ci][ti]['mse_finite_exact']    for ci in range(10)]) ,
                           'mse_weno5_exact'    :np.sum([error_log[ci][ti]['mse_weno5_exact']     for ci in range(10)])  }
    
error_log[-1] = error_log_all

### Errorr ratio

In [ ]:
#chunk = 0

diffusion_to_weno ={}
finite_to_weno ={}
diffusion_to_finite ={}
for ci in range(10):
    diffusion_to_weno[ci] = np.array ([ error_log[0][ti]['mse_diffusion_exact'] / error_log[0][ti]['mse_weno5_exact'] for ti in range(1000)])
    finite_to_weno[ci] = np.array ([ error_log[0][ti]['mse_finite_exact'] / error_log[0][ti]['mse_weno5_exact'] for ti in range(1000)])
    diffusion_to_finite[ci] = np.array ([ error_log[0][ti]['mse_diffusion_exact'] / error_log[0][ti]['mse_finite_exact'] for ti in range(1000)])

# all chunks
diffusion_to_weno[-1] = np.array ([ error_log[-1][ti]['mse_diffusion_exact'] / error_log[-1][ti]['mse_weno5_exact'] for ti in range(1000)])
finite_to_weno[-1] = np.array ([ error_log[-1][ti]['mse_finite_exact'] / error_log[-1][ti]['mse_weno5_exact'] for ti in range(1000)])
diffusion_to_finite[-1] = np.array ([ error_log[-1][ti]['mse_diffusion_exact'] / error_log[-1][ti]['mse_finite_exact'] for ti in range(1000)])


In [ ]:
diffusion_to_weno[0][diffusion_to_weno[0]<1].shape[0]

### Instances with lower error than WENO5 / FiniteNet

In [ ]:
print('Instances with lower error than WENO5 for chunk=0\t\t',diffusion_to_weno[0][diffusion_to_weno[0]<1].shape[0])
print('Instances with lower error than FiniteNet for chunk=0\t\t',diffusion_to_finite[0][diffusion_to_finite[0]<1].shape[0])

print('Instances with lower error than WENO5 for ALL chunks\t\t' , diffusion_to_weno[-1][diffusion_to_weno[-1]<1].shape[0])
print('Instances with lower error than FiniteNet for ALL chunks\t' , diffusion_to_finite[-1][diffusion_to_finite[-1]<1].shape[0])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,5))

plt.subplot(1,3,1)
plt.hist(diffusion_to_weno_all,bins=np.arange(0,5,0.05),alpha=0.5 ,label = '$DiffusionNet/WENO5$',color='red') 
plt.hist(finite_to_weno_all,bins=np.arange(0,5,0.05),alpha=0.5 ,label = '$FiniteNet/WENO5$',color='blue')
plt.hist(diffusion_to_finite_all,bins=np.arange(0,5,0.05),alpha=0.5 ,label = '$DiffusionNet/FiniteNet$',color='green')

plt.ylabel('$Frequency$');plt.xlabel('$Error  \ ratio$')
plt.title('$Error \  comparison \ for \ \Sigma C_i$')

plt.legend()

plt.subplot(1,3,2)

plt.hist(diffusion_to_weno[0],bins=np.arange(0,5,0.05),alpha=0.5 ,label = '$DiffusionNet/WENO5$',color='red') 
plt.hist(finite_to_weno[0],bins=np.arange(0,5,0.05),alpha=0.5 ,label = '$FiniteNet/WENO5$',color='blue')
plt.hist(diffusion_to_finite[0],bins=np.arange(0,5,0.05),alpha=0.5 ,label = '$DiffusionNet/FiniteNet$',color='green')

plt.ylabel('$Frequency$');plt.xlabel('$Error  \ ratio$')
plt.title('$Error \ comparison \  for \  C_1$')

plt.legend()


plt.subplot(1,3,3)
f_bars= [ finite_to_weno[j][finite_to_weno[j]<1].shape[0] for j in range(10) ] ; f_bars.append(finite_to_weno[-1][finite_to_weno[-1]<1].shape[0])
d_bars= [ diffusion_to_weno[j][diffusion_to_weno[j]<1].shape[0] for j in range(10) ] ; d_bars.append(diffusion_to_weno[-1][diffusion_to_weno[-1]<1].shape[0])
fd_bars= [ diffusion_to_finite[j][diffusion_to_finite[j]<1].shape[0] for j in range(10) ] ; fd_bars.append(diffusion_to_finite[-1][diffusion_to_finite[-1]<1].shape[0])

N = 11
finite = f_bars  #(error_finite_weno[0][error_finite_weno[0]<1].shape, 35, 30, 35, 27)
diffusion = d_bars
diffusionFinite=fd_bars

ind = np.arange(N) 
width = 0.25       
plt.bar(ind, finite, width, label='$DiffusionNet$',color='red',alpha=0.5)
plt.bar(ind + width, diffusion, width,label='$FiniteNet$',color='blue',alpha=0.5)
# plt.bar(ind + 2*width, diffusionFinite, width,label='$FiniteNet$',color='green',alpha=0.5)
plt.ylabel('$Frequency$')
plt.title('$Instances \ with error \ lower \ than \ WENO5$')

plt.xticks(ind + width / 2, ('$C1$','$C2$','$C3$','$C4$','$C5$','$C6$','$C7$','$C8$','$C9$','$C10$','$\Sigma C$'))
plt.legend(loc='center')


plt.tight_layout()

## Sample plot of Burgers figure

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import *

sample_slider = IntSlider(value=150,min=0,max=1000,step=1,description='Test #:',disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d',)



@interact(sample=sample_slider)
def plot_figure(sample):
    t= np.arange(0,1,0.01)
    plt.figure(figsize=(15,20))
    


    for i in range(1,11):

        plt.subplot(5,2,i)

        plt.plot(   t,test_data_exact[sample,i*10],'-k' ,linewidth=1)
        plt.scatter(t,test_data_finite[sample,i*10]    ,alpha=.75,color='red',marker='x')
        plt.scatter(t,test_data_weno5[sample,i*10]     ,alpha=.75,color='green',marker='+')
        plt.scatter(t,test_data_diffusion[sample,i*10] ,alpha=.75,color='blue',marker='1')

        plt.title(f'$Step={i*10} $')

        plt.tight_layout()
        plt.legend(['Exact', 
                    'FiniteNet  MSE={}e-6'.format(mse(test_data_finite[sample,i*10],test_data_exact[sample,i*10])  *1e6),
                    'WENO5     MSE={}e-6'.format(mse(test_data_weno5[sample,i*10],test_data_exact[sample,i*10])  *1e6), 
                    'DiffusionNet MSE={}e-6'.format( mse(test_data_diffusion[sample,i*10],test_data_exact[sample,i*10])  *1e6),
                   ])
    

# plt.savefig('dfn.svg')